<a href="https://colab.research.google.com/github/jmauriciolopez/Colab/blob/main/Llama_index_RAG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q llama-index==0.9.1 chromadb

In [ ]:
from llama_index.llms import OpenAI, Gemini
from llama_index import VectorStoreIndex, SimpleDirectoryReader
from llama_index.response.notebook_utils import display_response
import os
from getpass import getpass

## Configurando keys

In [ ]:
GOOGLE_API_KEY = getpass('Enter the secret value for Google Key: ')
os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY

Enter the secret value for Google Key: ··········


In [ ]:
OPENAI_API_KEY = getpass('Enter the secret value for OpenAI Key: ')
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

Enter the secret value for OpenAI Key: ··········


In [ ]:
resp = OpenAI().complete('Que tipo de animal es Leo en la pelicula?')
print(resp)

Leo es un león en la película.


In [ ]:
resp = Gemini().complete('Que tipo de animal es Leo en la pelicula?')
print(resp)

Leo no es un animal en la película. Leo es un personaje humano interpretado por Leonardo DiCaprio.


## RAG con Llama index

In [ ]:
### Cargando documentos
documents = SimpleDirectoryReader('src').load_data()

In [ ]:
documents

[Document(id_='25eaf008-b424-459f-918f-0d9b234ae31b', embedding=None, metadata={'file_path': 'src/Leo.txt', 'file_name': 'Leo.txt', 'file_type': 'text/plain', 'file_size': 76949, 'creation_date': '2024-01-11', 'last_modified_date': '2024-01-11', 'last_accessed_date': '2024-01-11'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={}, hash='26bac1d8ef1e3b0478febabfed94cc29823da9781035cc7e0625c1dadd9992ed', text="[male 1] Hey. You know the 5th grade starts tomorrow again.\n\n[male 2] What?! Does that mean we have to get up at 8:00?\n\n[male 1] Oh, calm down.\n\nYou slept 95% of the summer.\n\n[male 2] Yeah.\n\n[high-spirited song plays]\n\n♪ Last year of elementary school Last year of bein’ a kid ♪\n\n♪ Being on top ♪\n\n♪ I’m gonna go out swingin’ ♪\n\n♪ Toy soldier

In [ ]:
### Creando un indice con base a los documentos
index = VectorStoreIndex.from_documents(documents)
query_engine = index.as_query_engine()

In [ ]:
## Ejecutando inferencia con Rag
response = query_engine.query('Que tipo de animal es Leo y que tipo su amigo?')
display_response(response)

**`Final Response:`** Leo es un lagarto y su amigo es una tortuga.

## ¿Que paso adentro?

In [ ]:
response.source_nodes

In [ ]:
for _ in response.source_nodes:
    print('X'*50)
    print(_.text)

### Valores default - service_context

In [ ]:
index.service_context

In [ ]:
index.service_context.llm

In [ ]:
index.service_context.embed_model

In [ ]:
index.service_context.transformations

### Valores default - service_context

In [ ]:
index.storage_context

In [ ]:
index.storage_context.vector_stores

## Mejorando RAG

In [ ]:
import chromadb
from llama_index.vector_stores import ChromaVectorStore
from llama_index.embeddings import HuggingFaceEmbedding
from llama_index.storage.storage_context import StorageContext
from llama_index.service_context import ServiceContext
from llama_index.prompts import PromptTemplate

### Embeddings

In [ ]:
embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-base-en-v1.5")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/777 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/438M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/366 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

In [ ]:
test_emeds = embed_model.get_text_embedding("Que tipo de animal es Leo y que tipo su amigo?")
test_emeds

[-0.00570334680378437,
 -0.01947423443198204,
 -0.006114975083619356,
 0.019936738535761833,
 0.04330310598015785,
 0.01746615581214428,
 0.019677357748150826,
 0.02935454063117504,
 -0.0069770230911672115,
 -0.0027876971289515495,
 -0.03825373202562332,
 -0.0017076019430533051,
 -0.04513166844844818,
 -0.020971370860934258,
 -0.06812335550785065,
 0.03742658346891403,
 0.054611653089523315,
 0.04613518714904785,
 0.04661178216338158,
 -0.003186027752235532,
 -0.013806985691189766,
 -0.023338153958320618,
 -0.023487892001867294,
 0.02889632061123848,
 -0.010705280117690563,
 -0.02005843259394169,
 -0.007215529680252075,
 0.023810869082808495,
 -0.030661674216389656,
 0.003456145292147994,
 0.017111387103796005,
 -0.013116071932017803,
 0.01017843745648861,
 0.018162302672863007,
 -0.047868240624666214,
 0.019108984619379044,
 -0.006136000156402588,
 -0.053797680884599686,
 -0.002763370517641306,
 -0.013116060756146908,
 0.008164050057530403,
 0.030053621158003807,
 0.014281785115599632

### LLM

In [ ]:
llm = Gemini(temperature=0.2, model="gemini-pro")

### Database

In [ ]:
db = chromadb.PersistentClient(path="./chroma_db_HF")

# create collection
chroma_collection = db.get_or_create_collection("Leo_netflix")

# assign chroma as the vector_store to the context
vector_store = ChromaVectorStore(chroma_collection=chroma_collection)

## Creando index mejorado

In [ ]:
### Seteando el service context y el storage context cambiando los valores default
svc = ServiceContext.from_defaults(embed_model=embed_model,llm=llm)
stc = StorageContext.from_defaults(vector_store=vector_store)

In [ ]:
## Creando un nuevo index
index = VectorStoreIndex.from_documents(
    documents, storage_context=stc, service_context=svc
)

In [ ]:
query_engine = index.as_query_engine()

### Cambiemos el template del LLM

In [ ]:
template = (
    "Dado el contexto que te proporcionare responde las preguntas \n"
    "Contexto:\n"
    "################################\n"
    "{context_str}"
    "################################\n"
    "Responde en español como si fueras un pirata: {query_str}\n"
)
qa_template = PromptTemplate(template)
query_engine.update_prompts(
    {"response_synthesizer:text_qa_template": qa_template}
)

## Ejecutando el modelo con RAG

In [ ]:
response = query_engine.query('Que tipo de animal es Leo y que tipo su amigo?')
display_response(response)

**`Final Response:`** Leo es una lagartija y su amigo es una tortuga.

In [ ]:
response = query_engine.query('Cual es el sueño de Leo?')
display_response(response)

**`Final Response:`** ¡Arrr, marinero! El sueño de Leo es ver el mundo, navegar por los siete mares y explorar tierras lejanas. Quiere vivir aventuras y conocer nuevas culturas. ¡Es un espíritu libre que busca el horizonte!

In [ ]:
response = query_engine.query('Porque los niños son amigos de Leo')
display_response(response)

**`Final Response:`** ¡Arrr, camarada! Los niños son amigos de Leo porque él es un lagarto sabio y amable que siempre está dispuesto a ayudarlos. Leo los escucha cuando tienen problemas y les da consejos sabios. También los hace reír y los ayuda a sentirse mejor consigo mismos. ¡Leo es un verdadero tesoro para estos niños!